In [1]:
import pandas as pd
import numpy as np
import ast
import unicodedata
import re
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('/Users/ishaandawra/Desktop/Machine Learning Notes/Machine Learning Projects/LLM_SummerHome_Recommender/data/raw_listings.csv')
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1419.0,https://www.airbnb.com/rooms/1419,2.025060e+13,18/06/25,previous scrape,Beautiful home in amazing area!,"This large, family home is located in one of T...",The apartment is located in the Ossington stri...,https://a0.muscache.com/pictures/76206750/d643...,1565,https://www.airbnb.com/users/show/1565,Alexandra,08/08/08,"Vancouver, Canada","I live in Vancouver, Canada with my husband an...",NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/7aeea...,https://a0.muscache.com/im/pictures/user/7aeea...,Commercial Drive,1.0,2.0,"['email', 'phone']",t,t,Neighborhood highlights,Little Portugal,NaN,43.64590,-79.42423,Entire home,Entire home/apt,10,NaN,3 baths,5.0,NaN,"[""First aid kit"", ""Wifi"", ""Indoor fireplace"", ...",NaN,28,730,28,28,730,730,28.0,730.0,NaN,t,0,0,0,0,18/06/25,6,0,0,0,0,0,NaN,19/07/15,07/08/17,5.00,5.00,5.00,5.00,5.00,5.00,5.00,NaN,f,1,1,0,0,0.05
1,8077.0,https://www.airbnb.com/rooms/8077,2.025060e+13,18/06/25,previous scrape,Downtown Harbourfront Private Room,Guest room in a luxury condo with access to al...,NaN,https://a0.muscache.com/pictures/11780344/141c...,22795,https://www.airbnb.com/users/show/22795,Kathie & Larry,22/06/09,"Toronto, Canada",My husband and I have been airbnb host for alm...,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/9a077...,https://a0.muscache.com/im/pictures/user/9a077...,Harbourfront,2.0,3.0,"['email', 'phone']",t,f,NaN,Waterfront Communities-The Island,NaN,43.64080,-79.37673,Private room in rental unit,Private room,2,NaN,1.5 baths,NaN,NaN,"[""Wifi"", ""Pool"", ""Gym"", ""Smoke alarm"", ""Heatin...",NaN,180,365,180,180,365,365,180.0,365.0,NaN,NaN,0,0,0,0,18/06/25,169,0,0,0,0,0,NaN,20/08/09,27/08/13,4.84,4.81,4.89,4.87,4.90,4.92,4.83,NaN,f,2,1,1,0,0.88
2,26654.0,https://www.airbnb.com/rooms/26654,2.025060e+13,18/06/25,city scrape,"World Class @ CN Tower, convention centre, The...","CN Tower, TIFF Bell Lightbox, Metro Convention...",There's a reason they call it the Entertainmen...,https://a0.muscache.com/pictures/81811785/5dcd...,113345,https://www.airbnb.com/users/show/113345,Adela,25/04/10,NaN,Welcome to Toronto! \r\n\r\nAfter our first me...,within a day,90%,29%,t,https://a0.muscache.com/im/users/113345/profil...,https://a0.muscache.com/im/users/113345/profil...,Entertainment District,4.0,10.0,"['email', 'phone', 'work_email']",t,t,Neighborhood highlights,Waterfront Communities-The Island,NaN,43.64608,-79.39032,Entire condo,Entire home/apt,4,1.0,1 bath,2.0,2.0,"[""Wine glasses"", ""Shared sauna"", ""Paid parking...",$158.00,28,1125,28,28,1125,1125,28.0,1125.0,NaN,t,18,48,78,168

In [3]:
keep_columns = ['name','neighbourhood_cleansed','property_type','accommodates','amenities','price','minimum_nights','maximum_nights','review_scores_rating']

df = df[keep_columns]

df = df.rename(columns={
    'name': 'Name',
    'neighbourhood_cleansed': 'Location',
    'property_type': 'Property_type',
    'review_scores_rating': 'Review_rating',
    'amenities': 'Amenities',
    'accommodates': 'Accommodates',
    'minimum_nights': 'Min_nights',
    'maximum_nights': 'Max_nights',
    'price': 'Price'
})

In [4]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df.head()

,name,location,property_type,accommodates,amenities,price,min_nights,max_nights,review_rating
0,Beautiful home in amazing area!,Little Portugal,Entire home,10,"[""First aid kit"", ""Wifi"", ""Indoor fireplace"", ...",NaN,28,730,5.00
1,Downtown Harbourfront Private Room,Waterfront Communities-The Island,Private room in rental unit,2,"[""Wifi"", ""Pool"", ""Gym"", ""Smoke alarm"", ""Heatin...",NaN,180,365,4.84
2,"World Class @ CN Tower, convention centre, The...",Waterfront Communities-The Island,Entire condo,4,"[""Wine glasses"", ""Shared sauna"", ""Paid parking...",$158.00,28,1125,4.80
3,Executive Studio Unit- Ideal for One Person,South Riverdale,Entire rental unit,1,"[""Drying rack for clothing"", ""Wine glasses"", ""...",NaN,365,365,4.94
4,Downtown Toronto - Waterview Condo,Waterfront Communities-The Island,Entire rental unit,2,"[""Wifi"", ""Pool"", ""Gym"", ""Smoke alarm"", ""Heatin...",NaN,180,365,5.00


In [5]:
print(f"Original data shape: {df.shape}")

Original data shape: (21092, 9)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21092 entries, 0 to 21091
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           21092 non-null  object 
 1   location       21092 non-null  object 
 2   property_type  21092 non-null  object 
 3   accommodates   21092 non-null  int64  
 4   amenities      21092 non-null  object 
 5   price          15365 non-null  object 
 6   min_nights     21092 non-null  int64  
 7   max_nights     21092 non-null  int64  
 8   review_rating  16102 non-null  float64
dtypes: float64(1), int64(3), object(5)
memory usage: 1.4+ MB


In [7]:
# For our LLM project we need amenities to be in list format. 
# The amenities column is currently a string representation of a list. We will convert it to an actual list.
# We also need to handle trailing spaces and inconsistent capitalization.
# as.literal_eval is used here to convert the string representation of a list into an actual list.

def parse_amenities(x):
    if pd.isna(x) or x == '':
        return []
    try:
        return [item.strip().lower() for item in ast.literal_eval(x)]
    except (ValueError, SyntaxError):
        return [item.strip().lower() for item in x.split(',')]

df['amenities'] = df['amenities'].apply(parse_amenities)

# I first tried to use lists, but for LLM comma seperated strings are easier to handle.
# So we will convert the list of amenities back to a comma-separated string.
# Debugging
df['amenities'] = df['amenities'].apply(lambda x: ", ".join(x) if isinstance(x, list) else "")

In [8]:
# Name, Location, and Property_type columns need to be cleaned for better consistency.
# We will convert them to lowercase, strip leading/trailing spaces, and ensure they are in string format.

def handle_text_columns(col):
    return col.astype(str).str.strip().str.lower()

df['name'] = handle_text_columns(df['name'])
df['location'] = handle_text_columns(df['location'])
df['property_type'] = handle_text_columns(df['property_type'])

In [9]:
print(df['amenities'].head())

0    first aid kit, wifi, indoor fireplace, essenti...
1    wifi, pool, gym, smoke alarm, heating, elevato...
2    wine glasses, shared sauna, paid parking on pr...
3    drying rack for clothing, wine glasses, fast w...
4    wifi, pool, gym, smoke alarm, heating, kitchen...
Name: amenities, dtype: object


In [10]:
df['price'] = df['price'].astype(str).str.replace('[\$,]', '', regex=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [11]:
df.head()

,name,location,property_type,accommodates,amenities,price,min_nights,max_nights,review_rating
0,beautiful home in amazing area!,little portugal,entire home,10,"first aid kit, wifi, indoor fireplace, essenti...",NaN,28,730,5.00
1,downtown harbourfront private room,waterfront communities-the island,private room in rental unit,2,"wifi, pool, gym, smoke alarm, heating, elevato...",NaN,180,365,4.84
2,"world class @ cn tower, convention centre, the...",waterfront communities-the island,entire condo,4,"wine glasses, shared sauna, paid parking on pr...",158.0,28,1125,4.80
3,executive studio unit- ideal for one person,south riverdale,entire rental unit,1,"drying rack for clothing, wine glasses, fast w...",NaN,365,365,4.94
4,downtown toronto - waterview condo,waterfront communities-the island,entire rental unit,2,"wifi, pool, gym, smoke alarm, heating, kitchen...",NaN,180,365,5.00


In [12]:
df['accommodates'] = pd.to_numeric(df['accommodates'], errors='coerce')
df['min_nights'] = pd.to_numeric(df['min_nights'], errors='coerce')
df['max_nights'] = pd.to_numeric(df['max_nights'], errors='coerce')
df['review_rating'] = pd.to_numeric(df['review_rating'], errors='coerce')

In [13]:
print("Missing values per column:")
print(df.isna().sum())

Missing values per column:
name                0
location            0
property_type       0
accommodates        0
amenities           0
price            5727
min_nights          0
max_nights          0
review_rating    4990
dtype: int64


In [14]:
# I am dropping rows with price and review rating as missing values since price is a crucial feature for our recommender system.
df = df.dropna(subset=['price'])
df = df.dropna(subset=['review_rating'])

In [15]:
df.head()

,name,location,property_type,accommodates,amenities,price,min_nights,max_nights,review_rating
2,"world class @ cn tower, convention centre, the...",waterfront communities-the island,entire condo,4,"wine glasses, shared sauna, paid parking on pr...",158.0,28,1125,4.80
6,cozy & private basement apartment free parking,wexford/maryvale,entire bungalow,4,"lockbox, drying rack for clothing, wine glasse...",143.0,2,31,4.96
7,yonge & bloor studio skyline,rosedale-moore park,entire rental unit,2,"wifi, kitchen, hot water, free street parking,...",94.0,28,720,4.18
8,fountain view studio - eaton center,bay street corridor,entire condo,3,"single level home, wifi, gym, kitchen, hot wat...",120.0,28,1100,4.17
9,yonge & bloor lakeview master br,rosedale-moore park,private room in rental unit,1,"wifi, kitchen, shampoo, elevator, stove, keypa...",69.0,28,1125,4.40


In [16]:
duplicates = df.duplicated(subset=['name', 'location', 'property_type', 'amenities'], keep=False)

num_duplicates = duplicates.sum()
print(f"Number of duplicate listings by name and location: {num_duplicates}")

if num_duplicates > 0:
    print("Duplicate listings:")
    print(df[duplicates].sort_values(['name', 'location']))
else:
    print("No duplicate listings found")

Number of duplicate listings by name and location: 21
Duplicate listings:
                                                    name  \
10275              *lovely 2 bdr unit/ parking+ subway *   
17489              *lovely 2 bdr unit/ parking+ subway *   
13374    *monthly* the pearl room | private room w/ desk   
16331    *monthly* the pearl room | private room w/ desk   
16151               cozy private room @ downtown toronto   
16152               cozy private room @ downtown toronto   
12838                            cozy, clean, convenient   
12840                            cozy, clean, convenient   
12842                            cozy, clean, convenient   
20695                       modern 1-br condo dt toronto   
20725                       modern 1-br condo dt toronto   
18749  prime location! city view condo in dundas square!   
20150  prime location! city view condo in dundas square!   
6254     private room & bath in charming victorian house   
6255     private room & ba

In [17]:
df = df[~duplicates].reset_index(drop=True)

print(f"After dropping all duplicates, dataset now has {len(df)} rows.")

After dropping all duplicates, dataset now has 12016 rows.


In [18]:
# We normalized unicode characters, removed non-ASCII characters, and stripped leading/trailing spaces from text data.
# This makes text consistent and easier for computers and LLMs to read.
# Since the dataset is very large, I did this step only to avoid performance issues.

def clean_text_data(text):
    if pd.isna(text):
        return ""
    text = unicodedata.normalize('NFKD', str(text))
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(r'[^\x20-\x7E]', '', text)
    text = text.strip()
    return text


for col in ['name', 'location', 'property_type']:
    df[col] = df[col].apply(clean_text_data)


df[['name', 'location', 'property_type','amenities']].head(3)

,name,location,property_type,amenities
0,"world class @ cn tower, convention centre, the...",waterfront communities-the island,entire condo,"wine glasses, shared sauna, paid parking on pr..."
1,cozy & private basement apartment free parking,wexford/maryvale,entire bungalow,"lockbox, drying rack for clothing, wine glasse..."
2,yonge & bloor studio skyline,rosedale-moore park,entire rental unit,"wifi, kitchen, hot water, free street parking,..."


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12016 entries, 0 to 12015
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           12016 non-null  object 
 1   location       12016 non-null  object 
 2   property_type  12016 non-null  object 
 3   accommodates   12016 non-null  int64  
 4   amenities      12016 non-null  object 
 5   price          12016 non-null  float64
 6   min_nights     12016 non-null  int64  
 7   max_nights     12016 non-null  int64  
 8   review_rating  12016 non-null  float64
dtypes: float64(2), int64(3), object(4)
memory usage: 845.0+ KB


In [20]:
df.head()

,name,location,property_type,accommodates,amenities,price,min_nights,max_nights,review_rating
0,"world class @ cn tower, convention centre, the...",waterfront communities-the island,entire condo,4,"wine glasses, shared sauna, paid parking on pr...",158.0,28,1125,4.80
1,cozy & private basement apartment free parking,wexford/maryvale,entire bungalow,4,"lockbox, drying rack for clothing, wine glasse...",143.0,2,31,4.96
2,yonge & bloor studio skyline,rosedale-moore park,entire rental unit,2,"wifi, kitchen, hot water, free street parking,...",94.0,28,720,4.18
3,fountain view studio - eaton center,bay street corridor,entire condo,3,"single level home, wifi, gym, kitchen, hot wat...",120.0,28,1100,4.17
4,yonge & bloor lakeview master br,rosedale-moore park,private room in rental unit,1,"wifi, kitchen, shampoo, elevator, stove, keypa...",69.0,28,1125,4.40


In [21]:
# sanity check
invalid_ratings = df[(df['review_rating']<0) & (df['review_rating']>5)]
print(invalid_ratings)

Empty DataFrame
Columns: [name, location, property_type, accommodates, amenities, price, min_nights, max_nights, review_rating]
Index: []


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12016 entries, 0 to 12015
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           12016 non-null  object 
 1   location       12016 non-null  object 
 2   property_type  12016 non-null  object 
 3   accommodates   12016 non-null  int64  
 4   amenities      12016 non-null  object 
 5   price          12016 non-null  float64
 6   min_nights     12016 non-null  int64  
 7   max_nights     12016 non-null  int64  
 8   review_rating  12016 non-null  float64
dtypes: float64(2), int64(3), object(4)
memory usage: 845.0+ KB


In [23]:
tags_list = ["cozy", "lakeview", "downtown", "beach", "studio", "condo", "apartment", "basement", "private room", "suite", "loft", "modern", "spacious", "luxury", "garden", "pet friendly", "pets", "family-friendly", "parking", "quiet", "city", "mountain", "waterfront", "historic", "furnished", "dog friendly", "dogs", "central", "charming", "bright", "renovated", "entertainment district", "high park", "annex", "little italy", "uptown", "penthouse", "shared", "accessible", "clean", "stylish", "private", "near subway", "walk to", "views", "safe", "secure", "budget", "luxurious", "parking included", "free parking", "wifi", "internet", "flat", "happy", "peaceful", "stylish", "office", "balcony", "colourful", "creative", "oasis", "cottage", "getaway", "uber", "paradise", "sunny", "reno'd", "greenhouse", "pad", "away", "serviced", "deluxe", "king", "garage", "premium", "view", "friendly", "kipling", "palmerston", "duwest", "bath", "spectacular", "annex", "beaches", "coach", "lrgl", "waterfront", "master", "trinity-bellwoods", "courtyard", "yoga", "massage", "contract", "best", "city"]

def extract_tags_str(row):
    text = (str(row['name']) + ' ' + str(row['location'])).lower()
    unique_tags = set(tag for tag in tags_list if tag in text)
    return ", ".join(sorted(unique_tags)) if unique_tags else ""


df['extracted_tags'] = df.apply(extract_tags_str, axis=1)


# This code extracts tags from the 'name' and 'location' columns of the dataFrame based on a predefined list of tags. Predefined list of tags are extracted from the dataset and are used to identify relevant tags in the listings.

# Initially, the tags extracted were having a lot of duplicates so I used a set to remove duplicates.
# Then tags are returned as a comma-separated string for easier handling in the recommender system.

In [24]:
df.rename(columns={'extracted_tags': 'tags'}, inplace=True)
df.head()

,name,location,property_type,accommodates,amenities,price,min_nights,max_nights,review_rating,tags
0,"world class @ cn tower, convention centre, the...",waterfront communities-the island,entire condo,4,"wine glasses, shared sauna, paid parking on pr...",158.0,28,1125,4.80,waterfront
1,cozy & private basement apartment free parking,wexford/maryvale,entire bungalow,4,"lockbox, drying rack for clothing, wine glasse...",143.0,2,31,4.96,"apartment, basement, cozy, free parking, king,..."
2,yonge & bloor studio skyline,rosedale-moore park,entire rental unit,2,"wifi, kitchen, hot water, free street parking,...",94.0,28,720,4.18,studio
3,fountain view studio - eaton center,bay street corridor,entire condo,3,"single level home, wifi, gym, kitchen, hot wat...",120.0,28,1100,4.17,"studio, view"
4,yonge & bloor lakeview master br,rosedale-moore park,private room in rental unit,1,"wifi, kitchen, shampoo, elevator, stove, keypa...",69.0,28,1125,4.40,"lakeview, master, view"


In [25]:
print("Missing values per column, sanity check for extracted_tags:")
print(df.isna().sum())

Missing values per column, sanity check for extracted_tags:
name             0
location         0
property_type    0
accommodates     0
amenities        0
price            0
min_nights       0
max_nights       0
review_rating    0
tags             0
dtype: int64


In [26]:
# Saving a sample of cleaned data to a new CSV file - Starting with 200 rows for testing purposes.

sample_df = df.sample(n=200, random_state=42).reset_index(drop=True)

sample_df.to_csv('/Users/ishaandawra/Desktop/Machine Learning Notes/Machine Learning Projects/LLM_SummerHome_Recommender/data/cleaned_listings.csv', index=False)
print("Cleaned data saved to 'cleaned_listings.csv'")

Cleaned data saved to 'cleaned_listings.csv'
